#SQLCoder2
Run the cells below to run inference on our text-to-SQL LLM: SQLCoder2. This loads the model with int4 quantization. You can use a larger GPU to load the int8 and float16 variants

##Setup

In [ ]:
!pip install torch transformers bitsandbytes accelerate sqlparse

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
torch.cuda.is_available()

##Download the Model
Use an A100 on Colab Pro (or any system with >30GB VRAM on your own machine) to load this in bf16. If unavailable, use a GPU with minimum 20GB VRAM to load this in 8bit, or with minimum 12GB of VRAM to load in 4bit. On Colab, it works with a V100 but crashes on a T4.

Downloading the model and then loading it to memory step takes around 10 minutes the first time. So please be patient :)

In [ ]:
model_name = "codellama/CodeLlama-34b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    # load_in_8bit=True,
    load_in_4bit=True,
    device_map="auto",
    use_cache=True,
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import sqlparse

# Function to extract question and schema from file content
def extract_question_schema(content):
    split_index = content.index("CREATE")
    question = content[:split_index].strip()
    schema = content[split_index:].strip()
    return question, schema

# Function to generate SQL query
def generate_sql(question, schema):
        prompt = """### Task
    Generate a SQL query to answer the following question:
    `{question}`

    ### Database Schema
    This query will run on a database whose schema is represented in this string:
    `{schema}`
    ### SQL
    Given the database schema, here is the SQL query that answers `{question}`:
    ```sql
    """.format(question=question)

    eos_token_id = tokenizer.eos_token_id

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=eos_token_id,
        pad_token_id=eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,

    )

    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service


    print(sqlparse.format(outputs[0].split("```sql")[-1], reindent=True))
    return generated_sql

input_folder_path = 'data/sqlcoder/question_and_schema/'
output_folder_path = 'data/sqlcoder/queries/'

for file_name in os.listdir(input_folder_path):
    if file_name.endswith('.txt'):
        with open(os.path.join(input_folder_path, file_name), 'r') as file:
            content = file.read()

            # Extract question and schema
            question, schema = extract_question_schema(content)

            # Generate SQL query
            generated_sql = generate_sql(question, schema)

            # Output the results
            output_file_name = f"{file_name}_output.sql"
            with open(os.path.join(output_folder_path, output_file_name), 'w') as output_file:
                output_file.write(generated_sql)
